# Implementing binary decision trees

In [1]:
#Libraries Import
import json
import string
import numpy as np
import pandas as pd
from math import exp
from math import sqrt
import matplotlib.pyplot as plt
pd.set_option("Chained_Assignment",None)

In [2]:
#read dataframe
loans=pd.read_csv("lending-club-data.csv",low_memory=False)

In [3]:
#we reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop('bad_loans',axis=1)

In [4]:
# using 4 categorical features only
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

In [5]:
#creating dummy variables
col_cat_names=[]
[col_cat_names.append(x) for x,y in zip(loans.columns,loans.dtypes) if y==object]

for attr in col_cat_names:
    loans = loans.merge(pd.get_dummies(loans[attr], prefix=attr), left_index=True, right_index=True)
    loans.drop(attr,axis=1,inplace=True)

In [6]:
#Let's see what the feature columns look like now:
features = loans.columns.tolist()
features.remove('safe_loans')  # Remove the response variable
features

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'term_ 36 months',
 'term_ 60 months',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'emp_length_1 year',
 'emp_length_10+ years',
 'emp_length_2 years',
 'emp_length_3 years',
 'emp_length_4 years',
 'emp_length_5 years',
 'emp_length_6 years',
 'emp_length_7 years',
 'emp_length_8 years',
 'emp_length_9 years',
 'emp_length_< 1 year']

In [7]:
print ("Number of features (after binarizing categorical variables) = %s" % len(features))

Number of features (after binarizing categorical variables) = 24


In [8]:
with open('module-5-assignment-2-train-idx.json') as train_data_file:    
    train_idx  = json.load(train_data_file)
with open('module-5-assignment-2-test-idx.json') as test_data_file:    
    test_idx = json.load(test_data_file)

train_data = loans.iloc[train_idx]
test_data = loans.iloc[test_idx]

In [9]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # safe loans
    safe_loans = np.asarray(labels_in_node==1,dtype=int).sum()
    
    # risky loans
    risky_loans = np.asarray(labels_in_node==-1,dtype=int).sum()
    
    # Return the number of mistakes that the majority classifier makes.
    minimum=min(safe_loans,risky_loans)
    
    return minimum

In [10]:
# Test case 1
example_labels = np.asarray([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print ('Test passed!')
else:
    print ('Test 1 failed... try again!')

# Test case 2
example_labels = np.asarray([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print ('Test passed!')
else:
    print ('Test 2 failed... try again!')
    
# Test case 3
example_labels = np.asarray([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print ('Test passed!')
else:
    print ('Test 3 failed... try again!')

Test passed!
Test passed!
Test passed!


In [11]:
def best_splitting_feature(data, features, target):
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        right_split = data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        left_mistakes = intermediate_node_num_mistakes(left_split[target])            

        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes(right_split[target])  
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes + right_mistakes) / num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        if error < best_error:
            best_feature = feature
            best_error = error
    
    return best_feature # Return the best feature we found

In [12]:

#To test your best_splitting_feature function, run the following code:


if best_splitting_feature(train_data, features, 'safe_loans') == 'term_ 36 months':
    print( 'Test passed!')
else:
    print ('Test failed... try again!')

Test passed!


In [13]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True
           }   
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] =  1        
    else:
        leaf['prediction'] = -1       
        
    # Return the leaf node        
    return leaf 

In [14]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    if intermediate_node_num_mistakes(target_values) == 0:  
        print ("Stopping condition 1 reached.")     
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if remaining_features == [] :   
        print ("Stopping condition 2 reached.")
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth: 
        print ("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    splitting_feature=best_splitting_feature(data, remaining_features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    remaining_features.remove(splitting_feature)
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(right_split[target])

        
    # Repeat (recurse) on left and right subtrees
    
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    right_tree =  decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth) 

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [15]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return (1 + count_nodes(tree['left']) + count_nodes(tree['right']))

In [16]:
small_data_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 3)
if count_nodes(small_data_decision_tree) == 13:
    print ('Test passed!')
else:
    print ('Test failed... try again!')
    print ('Number of nodes found                :', count_nodes(small_data_decision_tree))
    print ('Number of nodes that should be there : 13' )

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 3 (1048 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length_< 1 year. (90, 11)
--------------------------------------------------------------------
Subtree, depth = 3 (90 data p

In [17]:
my_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
R

In [18]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print( "At leaf, predicting %s" % tree['prediction'])
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [19]:
print(test_data.iloc[0])
print( 'Predicted class: %s ' % classify(my_decision_tree, test_data.iloc[0]))

safe_loans                -1
grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
Name: 24, dtype: int64
Predicted class: -1 


In [20]:
classify(my_decision_tree, test_data.iloc[0], annotate=True)

Split on term_ 36 months = 0
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
At leaf, predicting -1


-1

# QUIZ Implementing binary decision trees

#### Question 1
What was the feature that my_decision_tree first split on while making the prediction for test_data[0]?

__Ans__: term_ 36 months

#### Question 2
What was the first feature that lead to a right split of test_data[0]?

__Ans__: grade.D

#### Question 3
What was the last feature split on before reaching a leaf node for test_data[0]?

__Ans__: grade.D

#### Question 4
Rounded to 2nd decimal point (e.g. 0.76), what is the classification error of my_decision_tree on the test_data?


In [21]:
def evaluate_classification_error(tree, data,target):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x), axis=1)
    
    # Once you've made the predictions, calculate the classification error and return it
    error= np.asarray(data[target].values != np.array(prediction),dtype=int).sum()
    return round(error/len(data),2)

__Ans__:

In [22]:
evaluate_classification_error(my_decision_tree,test_data,"safe_loans")

0.38

#### Question 5
What is the feature that is used for the split at the root node?

__Ans__: term. 36 months

#### Question 6
What is the path of the first 3 feature splits considered along the left-most branch of my_decision_tree?

__Ans__: term. 36 months, grade.A, grade.B

#### Question 7
What is the path of the first 3 feature splits considered along the right-most branch of my_decision_tree?

__Ans__: term. 36 months, grade.D, no third feature because second split resulted in leaf